In [90]:
import quandl

import pandas_datareader as pdr
import pandas as pd
import numpy as np

quandl.ApiConfig.api_key = 'bNhmL6hk7a3boYd7Rbfz'

In [91]:
def get_all_symbols(to_list=True):
    '''Get all NASDAQ sybols'''
    from pandas_datareader.nasdaq_trader import get_nasdaq_symbols
    symbols = get_nasdaq_symbols()
    symbols = symbols.reset_index()
    symbols = symbols['Symbol']
    if to_list:
        return symbols.tolist()
    return symbols

slst = get_all_symbols()
universe = quandl.get_table('SHARADAR/SF1',calendardate='2018-12-31',ticker=slst)

In [161]:
def simple_pipe(df,condition_str,collist=None):
    '''Very simple version -- using query function in pd'''
    df = df.query(condition_str)
    if collist is not None:
        collist.append('ticker')
        df = df[collist].set_index('ticker')
    return df


df = universe
spr = simple_pipe(df,'currentratio >= currentratio.quantile(.75) | currentratio <= currentratio.quantile(.25)')  
base_universe = spr 

base_universe.tail()

,ticker,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,...,sharesbas,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital
None,,,,,,,,,,,,,,,,,,,,,
13,MMM,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-25,-6866000000,36500000000,37282000000,1.370900e+10,...,582287135,588500000,6.020000e+08,55.675,23792000000,0,1637000000,243000000,40.428,6.465000e+09
15,KO,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-24,-12814000000,83216000000,88242000000,2.493000e+10,...,4256513898,4259000000,4.299000e+09,7.480,61629000000,2674000000,1623000000,2765000000,14.470,-3.852000e+09
18,INTC,MRY,2018-12-31,2018-12-29,2018-12-29,2019-10-25,-974000000,127963000000,127693250000,2.878700e+10,...,4564000000,4611000000,4.701000e+09,15.365,91614000000,0,2264000000,6562000000,19.869,1.216100e+10
24,DD,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-31,-12394000000,187855000000,188029000000,1.266550e+11,...,764740291,766992000,7.718260e+08,112.096,138704000000,178000000,1489000000,4027000000,180.842,5.334300e+10
26,CSCO,MRY,2018-12-31,2018-07-28,2018-07-28,2019-11-19,-849000000,108784000000,121206000000,6.183700e+10,...,4702882494,4837000000,4.881000e+09,10.198,74526000000,3219000000,12929000000,9589000000,15.407,3.480200e+10


In [162]:
nest_dict = { 
    'FILT': {
        'cond1':df['workingcapital'] > 5000000,  
        'cond2':df['sps'] > 50,
        #'cond3':df['currentratio'] > 1.5, #top_25, -- used this in base_univ instead.
    },
    'COLS': { 
        'col1' : 'workingcapital',
        'col2': 'ticker',
        'col3':'sps',
    },
    
    'INDIC': { 
        'm1': df['currentratio'].quantile(.50),
        'tst': df['currentratio'] > nest_dict['INDIC']['m1'],
        't2': df['de'].mean()
        
    },
    'BASE':base_universe
}


def make_pipe(pipe_dict,ret_all=True):
    '''
    Create DF using pipeline dict -- 
    if ret_all is false, returns only explicit columsn in COLS
    '''
    f = pipe_dict['FILT']
    c = pipe_dict['COLS']
    ic = pipe_dict['INDIC']
    df = pipe_dict['BASE'] #Bingo! That's what was needed.
    
    #Add indicators
    for k,v in ic.items(): 
        df[str(k)] = v
        
    #Try filters? Think these worked?!
    for k, v in f.items():
        df = df[v]
        
    if ret_all is False:    
        cols = [i for i in c.values()]
        df = df[cols]
    df = df.set_index('ticker') 
    return df


pipe = make_pipe(nest_dict)
pipe

/Users/zoakes/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/zoakes/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,...,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,m1,tst,t2
ticker,,,,,,,,,,,,,,,,,,,,,
MMM,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-25,-6866000000,36500000000,37282000000,1.370900e+10,2.279100e+10,...,55.675,23792000000,0,1637000000,243000000,40.428,6.465000e+09,1.2085,True,13.013548
DD,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-31,-12394000000,187855000000,188029000000,1.266550e+11,6.120000e+10,...,112.096,138704000000,178000000,1489000000,4027000000,180.842,5.334300e+10,1.2085,True,13.013548


In [43]:
'''Create a merged df with OHLC'''

qsecs = universe['ticker'].tolist()

def merge_prices_funds(sec_list,df):
    #df = df.set_index('ticker')
    CLOSE_DICT = {}
    for sec in sec_list:
        temp = pdr.DataReader(sec,'yahoo','2019').tail()
        c = temp['Close'].iloc[-1]
        CLOSE_DICT[sec] = c
        df.loc[sec,'close'] = c #Very clean!
    return df

    
df2 = universe.set_index('ticker')
df2.loc['XOM','taxliabilities']



merge_prices_funds(qsecs,df2).head()

,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,...,shareswa,shareswadil,sps,tangibles,taxassets,taxexp,taxliabilities,tbvps,workingcapital,close
ticker,,,,,,,,,,,,,,,,,,,,,
XOM,MRY,2018-12-31,2018-12-31,2018-12-31,2019-11-06,-19564000000,346196000000,349610000000,4.797300e+10,2.982230e+11,...,4233807170,NaN,68.546,346196000000,0,9532000000,29856000000,81.769,-9.165000e+09,68.129997
WMT,MRY,2018-12-31,2019-01-31,2019-01-31,2019-09-06,-11542000000,219295000000,214216750000,6.189700e+10,1.573980e+11,...,2929000000,2.945000e+09,175.625,188114000000,0,4281000000,12409000000,64.225,-1.558000e+10,119.089996
VZ,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-30,2370000000,264829000000,264552500000,3.463600e+10,2.301930e+11,...,4128000000,4.132000e+09,31.701,136310000000,0,3584000000,33795000000,33.021,-3.294000e+09,60.240002
V,MRY,2018-12-31,2018-09-30,2018-09-30,2019-11-16,547000000,69225000000,68399000000,1.821600e+10,5.100900e+10,...,2049000000,2.586000e+09,10.058,26473000000,0,2505000000,4618000000,12.920,6.911000e+09,184.509995
UTX,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-25,-9333000000,134211000000,112374500000,3.550300e+10,9.870800e+10,...,800400000,8.101000e+08,83.085,59675000000,1646000000,2626000000,0,74.556,4.135000e+09,148.339996


In [193]:
pipe

,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,...,tbvps,workingcapital,m1,tst,t2,MA,SAR,STD,UBB,LBB
ticker,,,,,,,,,,,,,,,,,,,,,
MMM,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-25,-6866000000,36500000000,37282000000,1.370900e+10,2.279100e+10,...,40.428,6.465000e+09,1.2085,True,13.013548,169.412333,171.567110,3.281003,172.693335,166.131330
DD,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-31,-12394000000,187855000000,188029000000,1.266550e+11,6.120000e+10,...,180.842,5.334300e+10,1.2085,True,13.013548,67.746666,67.482881,2.585538,70.332205,65.161128


In [166]:

def add_technicals(pipe,pipe_dict=None): #Add a new section for indicators in pipe dict?
    '''Adds technicals to DFS dict, and Fundamentals DF'''
    pipe_secs = pipe.index
    dfs = {}
    
    for sec in pipe_secs:
        df = pdr.DataReader(str(sec),'yahoo',2019)
        df = pdr.DataReader(str(sec),'yahoo')
        df['SAR'] = ta.SAR(df['High'],df['Low'],.02,.2)
        df['MA'] = df['Close'].rolling(30).mean()
        df['STD'] = df['Close'].rolling(30).std() 
        df['UBB'] = df['MA'] + df['STD'] 
        df['LBB'] = df['MA'] - df['STD']
        
        #Add to pipe
        pipe.loc[sec,'MA'] = df['MA'].iloc[-1]
        pipe.loc[sec,'SAR'] = df['SAR'].iloc[-1]
        pipe.loc[sec,'STD'] = df['STD'].iloc[-1]
        pipe.loc[sec,'UBB'] = df['UBB'].iloc[-1]
        pipe.loc[sec,'LBB'] = df['LBB'].iloc[-1]
        
        #Create Proper Index
        df['ticker'] = str(sec)
        df = df.set_index('ticker') #.iloc[-1]
        dfs[sec] = df
    return pipe
            
        

add_technicals(pipe)

,dimension,calendardate,datekey,reportperiod,lastupdated,accoci,assets,assetsavg,assetsc,assetsnc,...,tbvps,workingcapital,m1,tst,t2,MA,SAR,STD,UBB,LBB
ticker,,,,,,,,,,,,,,,,,,,,,
MMM,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-25,-6866000000,36500000000,37282000000,1.370900e+10,2.279100e+10,...,40.428,6.465000e+09,1.2085,True,13.013548,169.412333,171.567110,3.281003,172.693335,166.131330
DD,MRY,2018-12-31,2018-12-31,2018-12-31,2019-10-31,-12394000000,187855000000,188029000000,1.266550e+11,6.120000e+10,...,180.842,5.334300e+10,1.2085,True,13.013548,67.746666,67.482881,2.585538,70.332205,65.161128


In [192]:
df = aapl

'''No clean way to do this inside dict without relying on new defs -- 
OR just NEED TAlib rather than manually done'''


tech_dict = {}


tech_dict = {
    'INDICS':{
        'MA': df['Close'].rolling(30).mean(),
        'STD': df['Close'].rolling(30).std(),
        #'UBB': tech_dict['INDICS']['MA'] + tech_dict['INDICS']['STD'], #May have trouble with this one -- undef 
    }
}
tech_dict['INDICS']['MA']



Date
2014-12-02           NaN
2014-12-03           NaN
2014-12-04           NaN
2014-12-05           NaN
2014-12-08           NaN
                 ...    
2019-11-22    252.002668
2019-11-25    253.019335
2019-11-26    253.985002
2019-11-27    255.100668
2019-11-29    256.166335
Name: Close, Length: 1258, dtype: float64